## Imports

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install -q tensorflow tensorflow-datasets matplotlib

In [0]:
from __future__ import absolute_import
import os
import math
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
from keras.datasets import cifar10

from keras import layers
from keras.layers import Add,Conv2D,Input,MaxPooling2D, BatchNormalization, Activation, Dropout, UpSampling2D, Dense
from keras.models import Model, Sequential
#from tensorflow.python.keras._impl.keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.regularizers import l2

from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from skimage import color

 
# Improve progress bar display
#import tqdm
#import tqdm.auto
#tqdm.tqdm = tqdm.auto.tqdm

picIteration = 1


## Variable declarations

In [0]:
#Paths etc.
trainPath = "drive/My Drive/Dataset/faceSmall"
testPath = "drive/My Drive/Dataset/faceSmall2"
resPath = "drive/My Drive/Dataset/Results"

#Parameters etc.
imgH = 32 #image height (that we want, not necessarily actual image size)
imgW = 32 #image width
batchSize = 9
epochs = 1900
learningRate = 0.0001





## Load data

In [26]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train[[1,2,3,4,5,90,92,93,23],:,:,:]
#X_test = X_test[[3,5,87,100,101,102,44,33,66,55,976],:,:,:]
#X_overtrained = X_train[[1,90,45,13],:,:,:]
#X_test = np.append(X_overtrained,X_test,axis=0)
#X_train = X_train[[1,2,3,4,5],:,:,:]
X_test = X_train

X_train.shape, X_test.shape

((9, 32, 32, 3), (9, 32, 32, 3))

In [0]:
datagenTrain = ImageDataGenerator(
                rescale=1/255,
                rotation_range=20,
                shear_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode= "nearest")

datagenTest = ImageDataGenerator(rescale=1/255) #kanske ta bort normaliseringen


## Lossfunction

In [0]:
#def lossFunction(y_true,y_pred):
  
#  res = []
#  print(y_pred.shape)
#  print(y_true.shape)
#  for i in range(batchSize):
#    res.append(1)
  
  
#  return res

## Build model 
50 layer ResNet, which has a slightly unusual architecture

![resnetsizes](https://i.stack.imgur.com/tkUYS.png)

In [0]:
def plussa(l1,l2):
  return Add([l1,l2])

In [0]:
#code for the resnet inspired by https://arxiv.org/pdf/1512.03385.pdf and https://engmrk.com/residual-networks-resnets/

def same_size_block(input, filterSizes):
  
  #save identity to be added in the end
  identity = input
  
  #first block
  layer = Conv2D(filters = filterSizes[0], kernel_size = (1,1),
                 strides = (1,1), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(input)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  
  #second block
  layer = Conv2D(filters = filterSizes[1], kernel_size = (3,3),
                 strides = (1,1), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(layer)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  #third block
  layer = Conv2D(filters = filterSizes[2], kernel_size = (1,1),
                 strides = (1,1), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(layer)
  
  #shortcut
  layer = Add()([layer,identity])
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  return layer

In [0]:
def downsample_block(input,filterSizes):
  
  #shortcut here need an extra layer, since it has the wrong dimensions and needs to be downsampled
  identity = input
  identity = Conv2D(filters = filterSizes[2], kernel_size = (1,1),
              strides = (2,2), padding = "same")(identity)
  
  #first block (notice stride = 2, to downsample.)
  layer = Conv2D(filters = filterSizes[0], kernel_size = (1,1),
                 strides = (2,2), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(input)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  #second block
  layer = Conv2D(filters = filterSizes[1], kernel_size = (3,3),
                 strides = (1,1), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(layer)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  #third block
  layer = Conv2D(filters = filterSizes[2], kernel_size = (1,1),
                 strides = (1,1), padding = "same", use_bias = True,
                 kernel_initializer = "he_normal", bias_initializer = "zeros")(layer)
  
  #shortcut
  layer = Add()([identity,layer])
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  return layer

In [0]:
def upsampling_block(input, filterSize):
  layer = Conv2D(filters=filterSize, kernel_size=(3, 3), strides=(1, 1),
                  padding="same", use_bias=True,
                  kernel_initializer="he_normal", bias_initializer="zeros")(input)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  layer = UpSampling2D(size=(2, 2))(layer)
  
  return layer

In [0]:
def normal_block(input,filterSize):
  layer = Conv2D(filters=filterSize, kernel_size=(3, 3), strides=(1, 1),
                  padding="same", use_bias=True,
                  kernel_initializer="he_normal", bias_initializer="zeros")(input)
  layer = Activation("relu")(layer)
  layer = BatchNormalization()(layer)
  
  return layer
  

In [68]:
inputLayer = Input(shape=(imgH,imgW,1), name = "Input")

conv1 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros",
               name = "1-7x7")(inputLayer)
conv1 = Activation("relu", name = "1-relu")(conv1)
conv1 = BatchNormalization(name = "1-batchNorm")(conv1)

pool = MaxPooling2D((3,3),strides = 2, name = "2-pool")(conv1)

L2 = downsample_block(pool,[64,64,256])
L2 = same_size_block(L2,[64,64,256])
L2 = same_size_block(L2,[64,64,256])

L3 = downsample_block(L2,[128,128,512])
L3 = same_size_block(L3,[128,128,512])
L3 = same_size_block(L3,[128,128,512])
L3 = same_size_block(L3,[128,128,512])

L4 = downsample_block(L3,[256,256,1024])
L4 = same_size_block(L4,[256,256,1024])
L4 = same_size_block(L4,[256,256,1024])
L4 = same_size_block(L4,[256,256,1024])
L4 = same_size_block(L4,[256,256,1024])
L4 = same_size_block(L4,[256,256,1024])

L5 = downsample_block(L4, [512,512,2048])
L5 = same_size_block(L5, [512,512,2048])
L5 = same_size_block(L5, [512,512,2048])

#we have 6 downsamples (first thing, maxpool, 4 downsamples (L2,L3,L4,L5))
#so have to have 6 upsamples

up = upsampling_block(L5,2048)#1
up = normal_block(up,1024)
up = upsampling_block(up,512)#2
up = normal_block(up,256)
up = upsampling_block(up,128)#3
up = normal_block(up,64)
up = upsampling_block(up,32)#4


#Output.
output = Conv2D(filters=2, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros")(up)
output = Activation("tanh")(output)
output = UpSampling2D(size=(2, 2))(output)#6


model = Model(input = inputLayer, output = output)


optimizer = Adam(lr = learningRate)
model.compile(loss = "mse", optimizer = optimizer)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("In..., outputs=Tensor("up...)`


In [60]:
inputLayer = Input(shape=(imgH,imgW,1))

conv1 = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros"
               )(inputLayer)
conv1 = Activation("relu")(conv1)
conv1 = BatchNormalization()(conv1)

#Conv2.
conv2 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv1)
conv2 = Activation("relu")(conv2)
conv2 = BatchNormalization()(conv2)

#Conv3.
conv3 = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv2)
conv3 = Activation("relu")(conv3)
conv3 = BatchNormalization()(conv3)

#Conv4.
conv4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv3)
conv4 = Activation("relu")(conv4)
conv4 = BatchNormalization()(conv4)

#Conv5.
conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv4)
conv5 = Activation("relu")(conv5)
conv5 = BatchNormalization()(conv5)

#Conv6.
conv6 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv5)
conv6 = Activation("relu")(conv6)
conv6 = BatchNormalization()(conv6)

#Conv7.
conv7 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1),
               padding="same", dilation_rate=(2, 2), use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv6)
conv7 = Activation("relu")(conv7)
conv7 = BatchNormalization()(conv7)

#Conv8.
conv8 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1),
               padding="same", dilation_rate=(2, 2), use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv7)
conv8 = Activation("relu")(conv8)
conv8 = BatchNormalization()(conv8)

#Conv9.
conv9 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1),
               padding="same", use_bias=True,
               kernel_initializer="he_normal", bias_initializer="zeros")(conv8)
conv9 = Activation("relu")(conv9)
conv9 = BatchNormalization()(conv9)

#Conv10.
conv10 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="he_normal", bias_initializer="zeros")(conv9)
conv10 = Activation("relu")(conv10)
conv10 = BatchNormalization()(conv10)
conv10 = UpSampling2D(size=(2, 2))(conv10)

#Conv11.
conv11 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="he_normal", bias_initializer="zeros")(conv10)
conv11 = Activation("relu")(conv11)
conv11 = BatchNormalization()(conv11)

#Conv12.
conv12 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="he_normal", bias_initializer="zeros")(conv11)
conv12 = Activation("relu")(conv12)
conv12 = BatchNormalization()(conv12)
conv12 = UpSampling2D(size=(2, 2))(conv12)

#Conv13.
conv13 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="he_normal", bias_initializer="zeros")(conv12)
conv13 = Activation("relu")(conv13)
conv13 = BatchNormalization()(conv13)

#Output.
output = Conv2D(filters=2, kernel_size=(3, 3), strides=(1, 1),
                padding="same", use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros")(conv13)
output = Activation("tanh")(output)
output = UpSampling2D(size=(2, 2))(output)


model = Model(input = inputLayer, output = output)


optimizer = Adam(lr = learningRate)
model.compile(loss = "mse", optimizer = optimizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("up...)`


## Train model

In [0]:
def get_samples(datagen):
    for batch in datagen.flow(X_train,
                             batch_size= batchSize,
                             shuffle=True):
        #Convert to Lab color space, i.e. shape: (batch_size, height, width, 3).
        #Lab has 3 channels: grayscale, a and b.
        batch = color.rgb2lab(batch)
        #Input: grayscale images (channel 0).
        X_batch = batch[:, :, :, 0] #Shape: (batch_size, height, width).
        X_batch = X_batch[..., np.newaxis] #Shape: (batch_size, height, width, 1).
        #Targets: a and b color channels for the images (channel 1 and 2).
        Y_batch = batch[:, :, :, 1:]
        #Change Lab color space interval from [-128, 128] to [-1, 1].
        Y_batch = Y_batch / 128
        yield (X_batch, Y_batch)
        


In [71]:
num_training_samples = X_train.shape[0]
print(num_training_samples)
num_steps_per_epoch = math.floor(num_training_samples / batchSize)
history = model.fit_generator(generator = get_samples(datagenTrain), epochs= epochs, steps_per_epoch= num_steps_per_epoch)

9
Epoch 1/1900
1/1 [==============================] - 0s 230ms/step - loss: 0.0104
Epoch 2/1900
1/1 [==============================] - 0s 196ms/step - loss: 0.0132
Epoch 3/1900
1/1 [==============================] - 0s 195ms/step - loss: 0.0140
Epoch 4/1900
1/1 [==============================] - 0s 194ms/step - loss: 0.0109
Epoch 5/1900
1/1 [==============================] - 0s 191ms/step - loss: 0.0108
Epoch 6/1900
1/1 [==============================] - 0s 195ms/step - loss: 0.0107
Epoch 7/1900
1/1 [==============================] - 0s 187ms/step - loss: 0.0119
Epoch 8/1900
1/1 [==============================] - 0s 192ms/step - loss: 0.0111
Epoch 9/1900
1/1 [==============================] - 0s 192ms/step - loss: 0.0103
Epoch 10/1900
1/1 [==============================] - 0s 180ms/step - loss: 0.0109
Epoch 11/1900
1/1 [==============================] - 0s 178ms/step - loss: 0.0117
Epoch 12/1900
1/1 [==============================] - 0s 180ms/step - loss: 0.0112
Epoch 13/1900
1/1 [====

## Test the model

In [72]:
num_testing_samples = X_test.shape[0]
print(num_testing_samples)
testing_batch_size = 1
num_testing_batches = int(np.ceil(num_testing_samples / testing_batch_size))

batch_index = 0
for batch in datagenTest.flow(X_test,
                              batch_size=testing_batch_size,
                              shuffle=False):
  batch = color.rgb2lab(batch)
  X_batch = batch[:, :, :, 0]
  X_batch = X_batch[..., np.newaxis]
  Y_batch_predicted = model.predict(X_batch) * 128
  Y_batch_true = batch[:, :, :, 1:]
  
  lab_image_colorized = np.zeros((imgH, imgW, 3))
  #Set grayscale channel.
  lab_image_colorized[:, :, 0] = X_batch[0, :, :, 0]
  lab_image_grayscale = np.copy(lab_image_colorized)
  lab_image_true = np.copy(lab_image_colorized)
  #Set a and b channels.
  lab_image_colorized[:, :, 1:] = Y_batch_predicted[0, :, :, :]
  lab_image_true[:, :, 1:] = Y_batch_true[0, :, :, :]
  #Convert to rgb color space.
  rgb_image_grayscale = color.lab2rgb(lab_image_grayscale)
  rgb_image_colorized = color.lab2rgb(lab_image_colorized)
  rgb_image_true = color.lab2rgb(lab_image_true)
  #imsave("drive/My Drive/Dataset/Results/img_result" + str(batch_index) + "1epo_grayscale.png", rgb_image_grayscale)
  imsave("drive/My Drive/Dataset/Results/img_result" + str(batch_index) + "1epo_predicted.png", rgb_image_colorized)
  imsave("drive/My Drive/Dataset/Results/img_result" + str(batch_index) + "1epo_true.png", rgb_image_true)
  if (batch_index == num_testing_batches - 1):
      break
  batch_index = batch_index + 1




9


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
